## preprocesamiento dataset


Une el texto: Primero, toma el título y el resumen de cada investigación y los junta en un solo párrafo coherente.

Estandariza las categorías: Luego, revisa la columna que dice a qué especialidad médica pertenece cada investigación (ej. "Oncología", "cardiología", etc.) y la limpia para que todo esté en minúsculas y sin espacios raros. Así, se asegura de que no haya confusiones.

Traduce a números: Esta es la parte clave. Crea nuevas columnas, una para cada especialidad médica importante. Luego, para cada investigación, marca con un 1 si pertenece a esa especialidad y con un 0 si no. Esto es como pasar una lista de asistencia, marcando "presente" o "ausente" para cada categoría.

Guarda el resultado final: Finalmente, se queda solo con lo importante (el párrafo de texto y las nuevas columnas de 1s y 0s) y lo guarda todo en un archivo nuevo, ya limpio y listo para ser usado.

In [5]:
import pandas as pd

# Cargar dataset original
df = pd.read_csv(r"data\raw\challenge_data-18-ago.csv", sep=";")

# Función robusta para unir título y abstract
def unir_texto(title, abstract):
    title = str(title).strip()   # quitar espacios en los extremos
    abstract = str(abstract).strip()
    if title.endswith("."):      # si ya tiene punto final
        return title + " " + abstract
    else:                        # si no lo tiene, se lo agregamos
        return title + ". " + abstract

# Crear columna text
df["text"] = df.apply(lambda row: unir_texto(row["title"], row["abstract"]), axis=1)

# Normalizamos categorías
df["group"] = df["group"].str.lower().str.strip()

# Crear columnas multietiqueta
for label in ["cardiovascular", "hepatorenal", "neurological", "oncological"]:
    df[label] = df["group"].apply(lambda x: 1 if label in x else 0)

# Nos quedamos solo con lo necesario
df_final = df[["text", "cardiovascular", "hepatorenal", "neurological", "oncological"]]

# Exportar
df_final.to_csv("data\processed\dataset_preprocesado.csv", index=False, encoding="utf-8")

print("✅ Archivo procesado y guardado en data\processed\dataset_preprocesado.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data\\raw\\challenge_data-18-ago.csv'

## Separacion en entrenamiento, validacion, prueba
En resumen, este código es un planificador estratégico para dividir los datos. Su objetivo es asegurarse de que, cuando separemos los datos para entrenar y probar la inteligencia artificial, la división sea justa e inteligente, especialmente con los casos más raros.

Esto es lo que hace paso a paso:

Analiza las Combinaciones: Primero, lee el archivo de datos ya limpio. En lugar de mirar cada enfermedad por separado, se fija en las combinaciones de enfermedades que tiene cada investigación. Por ejemplo, identifica si un estudio trata solo de "cáncer", o si trata de "cáncer + neurología" al mismo tiempo. A cada combinación le pone una etiqueta única.

Cuenta los Grupos: Una vez que ha etiquetado todas las investigaciones, cuenta cuántos ejemplos hay de cada combinación. Básicamente, hace un inventario para saber qué tan comunes o raras son. Por ejemplo, podría descubrir que hay 500 estudios solo de "cáncer" pero solo 3 que son de "cáncer + neurología".

Crea un Plan de Reparto Inteligente (80/10/10): Esta es la parte más importante. Sabiendo cuántos ejemplos hay de cada tipo, diseña un plan para dividirlos en tres montones:

Entrenamiento (80%): El montón más grande, para que el modelo aprenda.
Validación (10%): Un pequeño montón para hacer pruebas durante el entrenamiento.
Prueba (10%): El montón final para ver qué tan bueno es el modelo.
La clave es que lo hace de forma muy cuidadosa. Si una combinación de enfermedades es muy rara (por ejemplo, solo hay 2 o 3 casos), se asegura de no "romper" ese grupito, poniéndolos juntos en el montón de entrenamiento para que el modelo al menos pueda aprender de ellos. Para las combinaciones más comunes, sí las reparte en los tres montones siguiendo la proporción 80/10/10.

Al final, lo que te muestra en pantalla no son los datos divididos todavía, sino el plan detallado de cómo se van a dividir, garantizando que cada combinación, por rara que sea, esté representada de forma justa.

In [ ]:
import pandas as pd
import numpy as np

# --- PASO 1: Leer el CSV y calcular la distribución de combinaciones ---

try:
    df_raw = pd.read_csv('data/dataset_preprocesado.csv')
    print("Paso 1: Archivo 'dataset_preprocesado.csv' cargado exitosamente.")
except FileNotFoundError:
    print("Error: El archivo 'dataset_preprocesado.csv' no fue encontrado.")
    exit()

# Columnas que representan las enfermedades
disease_columns = ['cardiovascular', 'hepatorenal', 'neurological', 'oncological']

# Función para crear la cadena de combinación para cada fila
def create_domain_string(row):
    present_diseases = [col for col in disease_columns if row[col] == 1]
    if present_diseases:
        return '+'.join(present_diseases)
    return 'none' # Etiqueta para filas sin ninguna de estas enfermedades

# Crear una nueva columna 'domain' con la combinación de enfermedades
df_raw['domain'] = df_raw.apply(create_domain_string, axis=1)

# Calcular la distribución (excluyendo las que no tienen ninguna enfermedad)
distribution_counts = df_raw[df_raw['domain'] != 'none']['domain'].value_counts().reset_index()
distribution_counts.columns = ['domain', 'count']
print("Paso 1: Distribución de combinaciones calculada.\n")


# --- PASO 2: Crear el plan de división (80/10/10) ---

def split_data_counts(row):
    """Calcula cuántas muestras van a train/val/test para una fila de la tabla de distribución."""
    count = row['count']
    if count == 1:
        return pd.Series([1, 0, 0], index=['train_count', 'val_count', 'test_count'])
    elif count == 2:
        return pd.Series([1, 1, 0], index=['train_count', 'val_count', 'test_count'])
    elif count == 3:
        return pd.Series([1, 1, 1], index=['train_count', 'val_count', 'test_count'])
    else:
        val_count = max(1, int(np.round(count * 0.1)))
        test_count = max(1, int(np.round(count * 0.1)))
        train_count = count - val_count - test_count
        return pd.Series([train_count, val_count, test_count], index=['train_count', 'val_count', 'test_count'])

# Aplicar la función para obtener el plan de división
split_plan = distribution_counts.copy()
split_counts = split_plan.apply(split_data_counts, axis=1)
split_plan = pd.concat([split_plan, split_counts], axis=1)

print("Paso 2: Plan de división 80/10/10 generado:")
print(split_plan.to_string())
print("\n")




Paso 1: Archivo 'dataset_preprocesado.csv' cargado exitosamente.
Paso 1: Distribución de combinaciones calculada.

Paso 2: Plan de división 80/10/10 generado:
                                                 domain  count  train_count  val_count  test_count
0                                          neurological   1058          846        106         106
1                                        cardiovascular    645          517         64          64
2                                           hepatorenal    533          427         53          53
3                           cardiovascular+neurological    308          246         31          31
4                                           oncological    237          189         24          24
5                              hepatorenal+neurological    202          162         20          20
6                            cardiovascular+hepatorenal    190          152         19          19
7                              neurological+oncol

## ejecucion de la separacion del dataset

En resumen, si el código anterior era el "planificador", este código es el "ejecutor". Toma el plan que se creó y lo lleva a cabo, repartiendo físicamente cada dato en el montón que le corresponde.

Así es como lo hace, paso a paso:

Repartir los Datos, Grupo por Grupo:

El código revisa el plan, fila por fila. Cada fila corresponde a una combinación de enfermedades (ej. "solo cáncer", o "cáncer + neurología").
Para cada combinación, busca todos los estudios que pertenecen a ese grupo en la tabla de datos original.
Luego, baraja aleatoriamente ese pequeño grupo de estudios. Esto es como barajar una parte de la baraja para que el reparto sea justo.
Finalmente, "corta" ese grupo barajado según los números del plan: los primeros van al montón de entrenamiento, los siguientes al de validación, y los últimos al de prueba.
Repite este proceso para todas y cada una de las combinaciones de enfermedades hasta que no queda ningún dato sin asignar.
Juntar y Mezclar los Montones Finales:

Después del reparto, ahora tiene un montón de pequeños "sub-grupos" para entrenamiento, validación y prueba.
Lo que hace a continuación es juntar todos los pedacitos de "entrenamiento" en un único y gran archivo de entrenamiento. Hace lo mismo para los otros dos.
Para terminar, vuelve a barajar cada uno de los tres montones grandes. Esto es muy importante para que el modelo de inteligencia artificial aprenda de forma desordenada y no, por ejemplo, viendo todos los casos de cáncer juntos.
Guardar los Resultados:

Una vez que los tres conjuntos de datos (entrenamiento, validación y prueba) están listos, limpios y bien mezclados, los guarda en tres archivos CSV separados.
Hacer una Última Verificación:

Al final, simplemente hace una suma rápida. Cuenta cuántos datos había al principio y cuántos hay en los tres nuevos archivos sumados, para asegurarse de que no se perdió ninguna investigación en el proceso.

In [ ]:
# --- PASO 3: Dividir el DataFrame original y crear los 3 conjuntos de datos ---

# Listas para almacenar los dataframes de cada conjunto
train_dfs, val_dfs, test_dfs = [], [], []

print("Paso 3: Iniciando la división de datos por cada combinación...")

# Iterar sobre el plan de división
for _, row in split_plan.iterrows():
    domain = row['domain']
    train_num = row['train_count']
    val_num = row['val_count']
    
    # Filtrar el dataframe original por la combinación actual
    domain_df = df_raw[df_raw['domain'] == domain]
    
    # Barajar aleatoriamente los datos de esta combinación para asegurar una división imparcial
    shuffled_domain_df = domain_df.sample(frac=1, random_state=42) # random_state para reproducibilidad
    
    # Cortar y asignar los datos a los conjuntos correspondientes
    train_slice = shuffled_domain_df.iloc[:train_num]
    val_slice = shuffled_domain_df.iloc[train_num:train_num + val_num]
    test_slice = shuffled_domain_df.iloc[train_num + val_num:]
    
    # Añadir las rebanadas a las listas
    train_dfs.append(train_slice)
    val_dfs.append(val_slice)
    test_dfs.append(test_slice)

# Combinar todas las rebanadas en tres DataFrames finales
train_set = pd.concat(train_dfs)
val_set = pd.concat(val_dfs)
test_set = pd.concat(test_dfs)

# Barajar los conjuntos finales para que las combinaciones no queden agrupadas
train_set = train_set.sample(frac=1, random_state=42).reset_index(drop=True)
val_set = val_set.sample(frac=1, random_state=42).reset_index(drop=True)
test_set = test_set.sample(frac=1, random_state=42).reset_index(drop=True)

# Eliminar la columna auxiliar 'domain' antes de guardar
train_set = train_set.drop(columns=['domain'])
val_set = val_set.drop(columns=['domain'])
test_set = test_set.drop(columns=['domain'])

print("Paso 3: División completada.\n")


# --- PASO 4: Guardar los archivos CSV ---

train_set.to_csv('data\processed\train_set.csv', index=False)
val_set.to_csv('data\processed\val_set.csv', index=False)
test_set.to_csv('data\processed\test_set.csv', index=False)

print("Paso 4: Archivos generados exitosamente:")
print("- train_set.csv")
print("- val_set.csv")
print("- test_set.csv\n")

# --- Verificación Final ---
print("--- Verificación Final ---")
print(f"Total de muestras originales: {len(df_raw[df_raw['domain'] != 'none'])}")
print(f"Muestras en train_set.csv: {len(train_set)}")
print(f"Muestras en val_set.csv:   {len(val_set)}")
print(f"Muestras en test_set.csv:  {len(test_set)}")
print(f"Suma total:                {len(train_set) + len(val_set) + len(test_set)}")
print("¡Proceso finalizado!")

Paso 3: Iniciando la división de datos por cada combinación...
Paso 3: División completada.

Paso 4: Archivos generados exitosamente:
- train_set.csv
- val_set.csv
- test_set.csv

--- Verificación Final ---
Total de muestras originales: 3565
Muestras en train_set.csv: 2851
Muestras en val_set.csv:   357
Muestras en test_set.csv:  357
Suma total:                3565
¡Proceso finalizado!


## Data Augmentation
En resumen, este código es un "estratega de datos". Su trabajo es analizar el conjunto de datos de entrenamiento, encontrar dónde hay desequilibrios (es decir, qué categorías tienen muy pocos ejemplos) y crear un plan detallado para solucionar ese problema generando datos nuevos. La técnica que planea usar es la "paráfrasis": tomar un artículo existente y reescribirlo para crear una nueva versión.

Aquí tienes el desglose de su estrategia:

Hacer un Inventario: Primero, abre el archivo de datos de entrenamiento y, una vez más, cuenta cuántos artículos hay para cada combinación específica de enfermedades. Este inventario es la base de toda su estrategia.

Diseñar un Plan de "Relleno" en Tres Fases: El código es muy inteligente y no trata a todos los artículos por igual. Sigue un plan jerárquico para decidir qué artículos duplicar:

Fase 1: Los más complejos y raros. Empieza con los artículos que cubren 3 o 4 enfermedades a la vez. Sabe que estos son muy valiosos. Su estrategia es nivelar los grupos: si la combinación más común de 3 enfermedades tiene 20 ejemplos, planea crear paráfrasis de las otras combinaciones de 3 enfermedades hasta que todas lleguen a 20.

Fase 2: Los de en medio. Luego, hace lo mismo con los artículos que cubren 2 enfermedades. Busca la combinación más popular y planea "rellenar" las demás para que alcancen ese mismo nivel.

Fase 3: El Gran Balance Final. Después de nivelar los grupos más complejos, mira el panorama general. Calcula el total de artículos para cada enfermedad individual (por ejemplo, cuántos estudios sobre "cáncer" hay en total, sin importar con qué esté combinado). Identifica la enfermedad con más ejemplos y establece ese número como el objetivo final para todas las demás. Luego, planea usar los artículos más simples (los de una sola enfermedad) como "relleno" para aumentar el conteo de las categorías que se quedaron atrás, hasta que todas estén equilibradas.

Presentar el Plan de Acción y los Resultados Esperados:

Al final, el código no ejecuta la creación de datos. En su lugar, te presenta el plan completo y listo para ejecutar. Te dice exactamente cuántos artículos de cada combinación específica necesitas parafrasear.
También te da una proyección del "antes y después". Te muestra una tabla comparando la cantidad de datos que tienes ahora con la cantidad que tendrás si sigues el plan, demostrando que al final todo quedará perfectamente balanceado.

In [19]:
import pandas as pd
import math

# --- PASO 0: Leer el dataset de entrenamiento y calcular la distribución ---

try:
    # Cargar el conjunto de entrenamiento generado previamente
    df_train = pd.read_csv('database/train_set.csv')
    print("Archivo 'train_set.csv' cargado correctamente.\n")

    # Columnas que representan las enfermedades
    disease_columns = ['cardiovascular', 'hepatorenal', 'neurological', 'oncological']

    # Función para crear la cadena de combinación para cada fila
    def create_domain_string(row):
        present_diseases = [col for col in disease_columns if row[col] == 1]
        if present_diseases:
            return '+'.join(present_diseases)
        return 'none'

    # Crear la columna 'domain' y calcular la distribución
    df_train['domain'] = df_train.apply(create_domain_string, axis=1)
    
    # Convertir la serie de value_counts a un diccionario
    combination_counts = df_train[df_train['domain'] != 'none']['domain'].value_counts().to_dict()

except FileNotFoundError:
    print("Error: No se encontró el archivo 'train_set.csv'.")
    print("Asegúrate de haber ejecutado el script anterior para generarlo.")
    exit()

# --- A PARTIR DE AQUÍ, ES EL SCRIPT DE OPTIMIZACIÓN QUE PROPORCIONASTE ---

# --- 1. DATOS INICIALES Y ESTRUCTURACIÓN ---
strata = {i: {} for i in range(1, 5)}
for combo, count in combination_counts.items():
    num_labels = combo.count('+') + 1
    strata[num_labels][combo] = count

# --- 2. PASO 1: REGLA FIJA PARA ARTÍCULOS DE 4 ETIQUETAS ---
paraphrase_plan = {combo: 0 for combo in combination_counts}

if 4 in strata and strata[4]:
    for combo, initial_count in strata[4].items():
        paraphrase_plan[combo] = initial_count

# --- 3. PASO 2: BALANCEAR ESTRATOS DE 3 Y 2 ETIQUETAS ---
for i in [3, 2]:
    stratum = strata.get(i, {})
    if not stratum: continue
    
    target_count = max(stratum.values())
    
    for combo, initial_count in stratum.items():
        if initial_count < target_count:
            needed_articles = target_count - initial_count
            calls_needed = math.ceil(needed_articles / 4)
            paraphrase_plan[combo] += min(calls_needed, initial_count)

# --- 4. CÁLCULO DE TOTALES INTERMEDIOS ---
categories = ["cardiovascular", "hepatorenal", "neurological", "oncological"]
intermediate_totals = {cat: 0 for cat in categories}
for combo, count in combination_counts.items():
    for cat in categories:
        if cat in combo:
            final_count = count + 4 * paraphrase_plan.get(combo, 0)
            intermediate_totals[cat] += final_count

# --- 5. PASO 3: USAR ARTÍCULOS DE 1 ETIQUETA COMO RELLENO ---
target_total = max(intermediate_totals.values())

for cat in categories:
    deficit = target_total - intermediate_totals[cat]
    if deficit > 0:
        calls_needed = math.ceil(deficit / 4)
        single_label_combo = cat
        initial_count = combination_counts.get(single_label_combo, 0)
        # Asegurarnos que el combo de una etiqueta exista antes de añadir al plan
        if single_label_combo in paraphrase_plan:
             paraphrase_plan[single_label_combo] += min(calls_needed, initial_count)
        else:
             paraphrase_plan[single_label_combo] = min(calls_needed, initial_count)

# --- 6. MOSTRAR RESULTADOS FINALES ---
final_plan = {k: v for k, v in paraphrase_plan.items() if v > 0}
total_api_calls = sum(final_plan.values())

print("--- 📋 PLAN DE BALANCEO HÍBRIDO FINAL (4 paráfrasis/llamada) 📋 ---")
sorted_plan = sorted(final_plan.items(), key=lambda item: item[1], reverse=True)
for combo, calls in sorted_plan:
    print(f"🔹 Parafrasear {calls} artículos de la combinación: '{combo}'")

print("\n-----------------------------------")
print(f"📞 Total de llamadas a la API necesarias: {total_api_calls}")
print("-----------------------------------")


print("\n--- 📈 RESULTADOS PROYECTADOS (TOTALES DE CATEGORÍA) 📈 ---")
final_totals = {cat: 0 for cat in categories}
for combo, count in combination_counts.items():
    for cat in categories:
        if cat in combo:
            final_totals[cat] += count + 4 * paraphrase_plan.get(combo, 0)

print(f"{'Categoría':<15} {'Inicial':<10} {'Final':<10}")
print("-" * 35)
for cat in categories:
    initial_total = sum(c for co, c in combination_counts.items() if cat in co)
    print(f"{cat.capitalize():<15} {initial_total:<10} {final_totals[cat]:<10}")

print("\n--- 📊 RESULTADOS PROYECTADOS (DISTRIBUCIÓN DE COMBINACIONES) 📊 ---")
print(f"{'Combinación':<55} {'Inicial':<10} {'Final':<10}")
print("-" * 75)
# Asegurarnos de mostrar todas las combinaciones originales
sorted_combos = sorted(combination_counts.keys(), key=lambda x: (x.count('+'), x))
for combo in sorted_combos:
    initial = combination_counts.get(combo, 0)
    final = initial + 4 * paraphrase_plan.get(combo, 0)
    print(f"{combo:<55} {initial:<10} {final:<10}")

Archivo 'train_set.csv' cargado correctamente.

--- 📋 PLAN DE BALANCEO HÍBRIDO FINAL (4 paráfrasis/llamada) 📋 ---
🔹 Parafrasear 163 artículos de la combinación: 'oncological'
🔹 Parafrasear 104 artículos de la combinación: 'hepatorenal'
🔹 Parafrasear 82 artículos de la combinación: 'cardiovascular'
🔹 Parafrasear 48 artículos de la combinación: 'cardiovascular+oncological'
🔹 Parafrasear 42 artículos de la combinación: 'hepatorenal+oncological'
🔹 Parafrasear 33 artículos de la combinación: 'neurological+oncological'
🔹 Parafrasear 24 artículos de la combinación: 'cardiovascular+hepatorenal'
🔹 Parafrasear 21 artículos de la combinación: 'hepatorenal+neurological'
🔹 Parafrasear 5 artículos de la combinación: 'cardiovascular+hepatorenal+neurological+oncological'
🔹 Parafrasear 5 artículos de la combinación: 'cardiovascular+hepatorenal+oncological'
🔹 Parafrasear 3 artículos de la combinación: 'cardiovascular+neurological+oncological'
🔹 Parafrasear 1 artículos de la combinación: 'hepatorenal+neu

## Generacion de datos con Gemini 

En resumen, si el código anterior era el "estratega", este es el "operario de la fábrica". Su única misión es ejecutar el plan de creación de datos, utilizando una potente inteligencia artificial (Gemini de Google) para hacer el trabajo pesado.

Aquí está el proceso explicado de manera sencilla:

Preparación y Verificación: Antes de encender las máquinas, el código hace una revisión de seguridad.

Primero, se asegura de que el "plan" creado en la celda anterior esté disponible. Si no lo encuentra, detiene todo.
Luego, busca la "llave" para acceder a la IA de Google (la API Key). Sin esta llave, no puede funcionar.
Redactar las Instrucciones para la IA: Esta es una parte fundamental. El código crea una plantilla de instrucciones muy clara y estricta para la inteligencia artificial. En esencia, le dice:

"Actúa como un experto editor de textos médicos."
"Te daré un título y un resumen. Tu trabajo es reescribirlo 4 veces de formas diferentes."
Reglas Cruciales: "No te inventes nada, no cambies los datos, mantén un tono profesional y, lo más importante, entrégame cada versión en este formato exacto: Título reescrito. Resumen reescrito."
"Dame el resultado final empaquetado en un formato específico (JSON) para que yo lo pueda entender fácilmente."
Poner en Marcha la Producción:

El código se convierte en un "jefe de producción". Revisa el plan y, para cada combinación de enfermedades, toma la cantidad exacta de artículos que necesita parafrasear.
Va uno por uno. Coge un artículo, lo mete en la plantilla de instrucciones y se lo envía a la IA de Gemini.
Mientras la IA trabaja, muestra una barra de progreso para que puedas ver en tiempo real cómo avanza la creación de los nuevos datos.
Cuando la IA devuelve las 4 nuevas versiones del texto, el código las recoge y las guarda en una lista temporal de "datos nuevos".
Por cortesía, hace una pequeña pausa entre cada petición para no sobrecargar el servicio de la IA.
Ensamblaje Final y Almacenamiento:

Una vez que ha pasado por todo el plan y ha generado cientos de nuevos artículos, toma la lista de "datos nuevos".
La combina con el conjunto de datos de entrenamiento original.
El resultado es un nuevo archivo de entrenamiento mucho más grande y balanceado.
Finalmente, guarda este archivo mejorado con el nombre train_set_expanded.csv, dejándolo listo para el paso final: entrenar un modelo mucho más robusto y preciso.

In [ ]:
# ==============================================================================
# SCRIPT DE EJECUCIÓN DEL PLAN DE BALANCEO
# (Esta celda asume que 'final_plan' y 'df_train' existen de la celda anterior)
# ==============================================================================
import google.generativeai as genai
import pandas as pd
from tqdm.auto import tqdm
from dotenv import load_dotenv 
import time
import os
import json

# --- 1. Verificación de Variables Previas ---
try:
    if 'final_plan' in locals() and 'df_train' in locals():
        print("✅ Variables 'final_plan' y 'df_train' encontradas de la celda anterior.")
        total_calls = sum(final_plan.values())
        print(f"Se ejecutarán {total_calls} llamadas a la API según el plan.")
    else:
        raise NameError
except NameError:
    print("❌ Error: Las variables 'final_plan' y 'df_train' no se encontraron.")
    print("Asegúrate de ejecutar la celda anterior que calcula el plan de optimización.")
    exit()

# --- 2. Configuración de la API ---
load_dotenv() 
try:
    API_KEY = os.environ.get('GEMINI_API_KEY')
    if API_KEY is None: raise ValueError
    print("🔑 API Key cargada desde la variable de entorno.")
except (ValueError, KeyError):
    API_KEY = "PEGA_TU_API_KEY_DE_GEMINI_AQUI" # 🚨 REEMPLAZA ESTO
    print("⚠️ API Key no encontrada. Usando la del script. ¡No olvides reemplazarla!")

genai.configure(api_key=API_KEY)

model = genai.GenerativeModel(
    'gemini-2.5-flash',
    generation_config={"response_mime_type": "application/json"}
)
print(f"🤖 Modelo seleccionado: {model.model_name}")

# --- 3. Definición del Prompt (MODIFICADO) ---
prompt_template = """
Act as an expert medical and scientific text editor.
Your task is to generate 4 distinct paraphrased versions of the provided research article title and abstract.

Crucial Instructions:
1. Preserve the factual meaning completely. Do not alter, invent, or omit any key medical entities, numerical results, or conclusions.
2. Maintain a formal, academic, and objective tone.
3. Each of the 4 versions must be linguistically different from the original and from each other.
4. STRUCTURE CONSTRAINTS (MANDATORY):
   - The source has two parts: Title and Abstract.
   - You MUST output each paraphrase as a single string strictly in this format:
     [Paraphrased Title]. [Paraphrased Abstract]
   - Use exactly one period followed by a single space as the only separator between title and abstract.
   - Do not add extra punctuation around the separator (no double periods, no colon).
   - Keep the title as a concise noun phrase (no trailing punctuation other than the required separator).
   - Do not move information between sections: title content stays in the title; abstract content stays in the abstract.
   - Do not add leading or trailing quotation marks in the final strings.
5. Format your response as a valid JSON object with a single key called "paraphrased_versions", which contains a list of the 4 paraphrased strings.

Original Title:
---
{title}
---

Original Abstract:
---
{abstract}
---

JSON Output:
"""

# --- 3.1 Utilidad para separar Título y Abstract del campo 'text' (NUEVA) ---
def split_title_abstract(txt: str):
    """
    Separa el primer '. ' como límite entre Título y Abstract.
    Si no encuentra '. ', intenta con el primer '.'.
    Si aún así no hay '.', retorna ('', txt) como fallback.
    """
    s = str(txt).strip()
    if ". " in s:
        t, a = s.split(". ", 1)
        return t.strip(), a.strip()
    idx = s.find(".")
    if idx != -1:
        return s[:idx].strip(), s[idx+1:].lstrip()
    return "", s

# --- 4. Bucle de Generación Guiado por el Plan ---
new_data = []

# (Opcional) PARA UNA PRUEBA RÁPIDA
#plan_de_prueba = {k: v for i, (k, v) in enumerate(final_plan.items()) if i < 1}
#plan_a_ejecutar = plan_de_prueba
plan_a_ejecutar = final_plan  # Para la ejecución completa

pbar = tqdm(total=sum(plan_a_ejecutar.values()), desc="🤖 Generando paráfrasis")

for combo, num_calls_for_combo in plan_a_ejecutar.items():
    # Selecciona los artículos de la combinación actual según lo planeado
    source_df = df_train[df_train['domain'] == combo].head(num_calls_for_combo)
    
    for index, row in source_df.iterrows():
        original_text = row['text']
        # --- (NUEVO) separar explícitamente Título y Abstract y pasarlos en el prompt ---
        t_part, a_part = split_title_abstract(original_text)
        prompt = prompt_template.format(title=t_part, abstract=a_part)
        
        try:
            response = model.generate_content(prompt)
            response_data = json.loads(response.text)
            paraphrased_versions = response_data["paraphrased_versions"]
            
            for version_text in paraphrased_versions:
                if isinstance(version_text, str) and version_text.strip():
                    # (Opcional) Validación ligera de estructura: debe contener '. ' al menos una vez
                    if ". " not in version_text:
                        # No forzamos corrección automática para no corromper contenido,
                        # solo registramos la anomalía si quieres monitorear.
                        pass
                    new_row = row.to_dict()
                    new_row['text'] = version_text.strip()
                    new_data.append(new_row)
            
            pbar.set_postfix_str(f"Éxito: '{combo[:25]}...'")
        except Exception as e:
            pbar.set_postfix_str(f"Error en '{combo[:20]}...': {e}")
            time.sleep(5)
            
        time.sleep(1.5)  # Pausa cortés entre peticiones
        pbar.update(1)

pbar.close()

# --- 5. Creación y Guardado del Dataset Expandido ---
if new_data:
    augmented_df = pd.DataFrame(new_data)
    
    # Prepara el df original (sin la columna 'domain' auxiliar)
    df_train_original = df_train.drop(columns=['domain'])
    # Prepara el df aumentado (sin la columna 'domain' auxiliar)
    if 'domain' in augmented_df.columns:
        augmented_df = augmented_df.drop(columns=['domain'])
    
    # Combina el original con el nuevo
    final_expanded_df = pd.concat([df_train_original, augmented_df], ignore_index=True)

    print(f"\n¡Proceso completado! ✅")
    print(f"Tamaño del dataset de entrenamiento original: {len(df_train_original)}")
    print(f"Número de muestras nuevas generadas: {len(augmented_df)}")
    print(f"Tamaño del dataset expandido: {len(final_expanded_df)}")
    
    # Guardado
    output_filename = "data\processed\train_set_expanded.csv"
    final_expanded_df.to_csv(output_filename, index=False)
    print(f"Nuevo dataset guardado como: '{output_filename}'")
else:
    print("\nNo se generaron nuevos datos. Revisa la barra de progreso por si hubo errores.")

✅ Variables 'final_plan' y 'df_train' encontradas de la celda anterior.
Se ejecutarán 531 llamadas a la API según el plan.
🔑 API Key cargada desde la variable de entorno.
🤖 Modelo seleccionado: models/gemini-2.5-flash


🤖 Generando paráfrasis: 100%|██████████| 531/531 [2:17:43<00:00, 15.56s/it, Éxito: 'cardiovascular+hepatorena...']                                      



¡Proceso completado! ✅
Tamaño del dataset de entrenamiento original: 2851
Número de muestras nuevas generadas: 2120
Tamaño del dataset expandido: 4971
Nuevo dataset guardado como: 'database/train_set_expanded.csv'


## generacion de datos de resultados

In [ ]:
# ==============================================================================
# CELDA FINAL: EVALUACIÓN EN TEST Y EXPORTACIÓN PARA DASHBOARD (VERSIÓN ESTÁNDAR)
# ==============================================================================
import os
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import json
from tqdm.auto import tqdm # Para una barra de progreso visual

print("--- Iniciando la evaluación final en el conjunto de prueba ---")


# --- 1. CONFIGURACIÓN DE RUTAS ---
# Rutas relativas para que funcione en cualquier PC con la misma estructura de carpetas
MODEL_DIR = "models/scibert_uncased"
TEST_DATA_PATH = "data\processed\test_set.csv"
OUTPUT_DIR = "data" # <-- CAMBIO: Simplificado para guardar en la carpeta principal de la base de datos
OUTPUT_FILENAME = os.path.join(OUTPUT_DIR, "test_predictions.csv") # <-- CAMBIO: Nombre del archivo de salida


# Asegúrate de que el directorio de salida exista
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 2. CARGA DE ACTIVOS (MODELO, TOKENIZADOR, DATOS) ---
print(f"Cargando modelo y tokenizador desde: {MODEL_DIR}")
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)

print(f"Cargando conjunto de prueba desde: {TEST_DATA_PATH}")
test_df = pd.read_csv(TEST_DATA_PATH)

# --- CAMBIO: Detectar etiquetas automáticamente y usar umbral por defecto ---
# Ya no cargamos el archivo 'best_thresholds.json'
DEFAULT_THRESHOLD = 0.5
LABEL_COLUMNS = [col for col in test_df.columns if col not in ['text', 'abstract', 'title', 'id']]

print(f"Etiquetas detectadas automáticamente: {LABEL_COLUMNS}")
print(f"Usando umbral por defecto para todas las etiquetas: {DEFAULT_THRESHOLD}")

# Configuración del dispositivo (GPU si está disponible, si no CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print(f"Modelo movido al dispositivo: {device}")


# --- 3. PREDICCIÓN EN LOTES (BATCH PREDICTION) ---
# (Esta sección no necesita cambios)
BATCH_SIZE = 32
all_probabilities = []
texts_to_predict = test_df['text'].tolist()

print(f"Iniciando predicción en lotes de tamaño {BATCH_SIZE}...")

for i in tqdm(range(0, len(texts_to_predict), BATCH_SIZE), desc="Procesando Lotes"):
    batch_texts = texts_to_predict[i:i + BATCH_SIZE]
    
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    probabilities = F.sigmoid(outputs.logits).cpu().numpy()
    all_probabilities.append(probabilities)

all_probabilities = np.vstack(all_probabilities)
print("Predicción en lotes completada.")


# --- 4. CONSTRUCCIÓN DEL DATAFRAME FINAL DE RESULTADOS ---
print("Construyendo el DataFrame de resultados...")
results_df = pd.DataFrame()
results_df['text'] = test_df['text']

for i, label in enumerate(LABEL_COLUMNS):
    results_df[f'{label}_true'] = test_df[label]
    results_df[f'{label}_prob'] = all_probabilities[:, i]
    # <-- CAMBIO: Se aplica el umbral por defecto en lugar de los umbrales optimizados
    results_df[f'{label}_pred'] = (all_probabilities[:, i] >= DEFAULT_THRESHOLD).astype(int)

print("DataFrame de resultados construido con éxito. Muestra:")
print(results_df.head())


# --- 5. GUARDAR EL ARCHIVO FINAL ---
results_df.to_csv(OUTPUT_FILENAME, index=False)
print(f"\n✅ ¡Éxito! Resultados guardados en: {OUTPUT_FILENAME}")
print("Este archivo ahora contiene todo lo necesario para alimentar las visualizaciones del dashboard.")

--- Iniciando la evaluación final en el conjunto de prueba ---
Cargando modelo y tokenizador desde: models/scibert_uncased
Cargando conjunto de prueba desde: database/test_set.csv
Etiquetas detectadas automáticamente: ['cardiovascular', 'hepatorenal', 'neurological', 'oncological']
Usando umbral por defecto para todas las etiquetas: 0.5
Modelo movido al dispositivo: cpu
Iniciando predicción en lotes de tamaño 32...


Procesando Lotes: 100%|██████████| 12/12 [09:36<00:00, 48.06s/it]


Predicción en lotes completada.
Construyendo el DataFrame de resultados...
DataFrame de resultados construido con éxito. Muestra:
                                                text  cardiovascular_true  \
0  An investigation of the pattern of kidney inju...                    0   
1  Effect of fucoidan treatment on collagenase-in...                    0   
2  Evaluation of the anticocaine monoclonal antib...                    0   
3  renal stone markers in valvular heart disease....                    1   
4  Evidence for a cholinergic role in haloperidol...                    0   

   cardiovascular_prob  cardiovascular_pred  hepatorenal_true  \
0             0.013677                    0                 1   
1             0.022620                    0                 0   
2             0.012825                    0                 1   
3             0.864140                    1                 1   
4             0.015429                    0                 0   

   hepatorenal_p